# Comprehensive Regression Modeling: Ames Housing Dataset

This notebook provides a complete end-to-end regression modeling pipeline using the Ames Housing dataset. We'll cover:

1. **Data Exploration & Understanding**
2. **Feature Engineering & Selection** 
3. **Model Selection & Training**
4. **Model Evaluation & Validation**
5. **Model Interpretation & Insights**
6. **Advanced Techniques & Optimization**

## 1. Setup and Data Loading

### 📦 Library Imports and Environment Setup

This cell imports all the essential libraries we'll need for our comprehensive regression modeling pipeline. Each library serves a specific purpose:

**Why these libraries?**
- **pandas & numpy**: Core data manipulation and numerical computation
- **matplotlib & seaborn**: Professional data visualization
- **scipy**: Statistical analysis and data transformations
- **scikit-learn**: Complete machine learning ecosystem
- **XGBoost/LightGBM**: State-of-the-art gradient boosting (if available)

The imports are organized by category to make the code readable and maintainable. We also configure display settings and visualization styles for consistent, professional output throughout the notebook.

In [ ]:
# Import essential libraries
# Data manipulation and analysis
import pandas as pd  # For data manipulation and analysis
import numpy as np   # For numerical computations and array operations

# Visualization libraries
import matplotlib.pyplot as plt  # For creating static plots and visualizations
import seaborn as sns            # For statistical data visualization with attractive defaults

# Statistical analysis
import warnings                           # To suppress unnecessary warnings
from scipy import stats                   # For statistical functions and tests
from scipy.stats import skew, boxcox_normmax  # For distribution analysis and transformation
from scipy.special import boxcox1p        # For Box-Cox transformation with lambda parameter

# Machine Learning libraries from scikit-learn
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
# train_test_split: Split data into training and testing sets
# cross_val_score: Perform k-fold cross-validation
# GridSearchCV/RandomizedSearchCV: Hyperparameter tuning methods

from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder
# StandardScaler: Standardize features to mean=0, std=1
# RobustScaler: Scale features using median and IQR (robust to outliers)
# LabelEncoder: Convert categorical labels to numerical values
# OneHotEncoder: Create binary columns for categorical variables

from sklearn.feature_selection import SelectKBest, f_regression, RFE, SelectFromModel
# SelectKBest: Select k highest scoring features
# f_regression: F-statistic for regression feature selection
# RFE: Recursive Feature Elimination
# SelectFromModel: Select features based on model coefficients/importance

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
# LinearRegression: Basic linear regression without regularization
# Ridge: L2 regularization to prevent overfitting
# Lasso: L1 regularization for feature selection
# ElasticNet: Combines L1 and L2 regularization

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
# RandomForest: Ensemble of decision trees with bagging
# GradientBoosting: Sequential ensemble that corrects previous model errors
# ExtraTrees: Extremely randomized trees for faster training

from sklearn.svm import SVR                    # Support Vector Regression
from sklearn.neighbors import KNeighborsRegressor  # k-Nearest Neighbors regression

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# Evaluation metrics for regression models

from sklearn.pipeline import Pipeline          # For creating ML pipelines
from sklearn.compose import ColumnTransformer  # For applying different transformations to different columns

# Advanced gradient boosting libraries (optional but powerful)
try:
    import xgboost as xgb
    xgb_available = True
    print("XGBoost imported successfully - enables powerful gradient boosting")
except ImportError:
    print("XGBoost not available. Install with: pip install xgboost")
    xgb_available = False

try:
    import lightgbm as lgb
    lgb_available = True
    print("LightGBM imported successfully - enables fast gradient boosting")
except ImportError:
    print("LightGBM not available. Install with: pip install lightgbm")
    lgb_available = False

# Configure environment settings
warnings.filterwarnings('ignore')  # Suppress warnings for cleaner output

# Set consistent visualization style
plt.style.use('seaborn-v0_8')  # Modern, clean plotting style
sns.set_palette("husl")         # Use attractive color palette

# Configure pandas display options for better data exploration
pd.set_option('display.max_columns', None)  # Show all columns when displaying DataFrames
pd.set_option('display.max_rows', 100)      # Show up to 100 rows

print("✅ All libraries imported successfully!")
print(f"XGBoost available: {xgb_available}")
print(f"LightGBM available: {lgb_available}")
print("\n📚 Libraries loaded for comprehensive regression modeling:")
print("   • Data processing: pandas, numpy")
print("   • Visualization: matplotlib, seaborn")
print("   • ML algorithms: 9+ regression models from scikit-learn")
print("   • Feature engineering: preprocessing and selection tools")
print("   • Model evaluation: comprehensive metrics and validation")

In [ ]:
# Load the Ames Housing dataset and perform initial inspection
# The Ames Housing dataset is a comprehensive dataset of residential properties in Ames, Iowa
# It contains detailed information about house characteristics and sale prices
df = pd.read_csv('datasets/AmesHousing.csv')

# Display basic dataset information
print(f"🏠 Ames Housing Dataset loaded successfully!")
print(f"Shape: {df.shape} (rows × columns)")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Perform initial data quality checks
print("\n=== DATASET OVERVIEW ===")
print(f"Total samples: {df.shape[0]:,}")           # Number of house records
print(f"Total features: {df.shape[1]}")            # Number of variables/columns
print(f"Duplicate rows: {df.duplicated().sum()}")  # Check for exact duplicates

# Display first few rows to understand data structure
print("\n=== SAMPLE DATA (First 3 Rows) ===")
print("This gives us a preview of the data structure and types:")
display(df.head(3))

# Quick info about data types
print(f"\n=== DATA TYPE SUMMARY ===")
print(f"Numerical columns: {len(df.select_dtypes(include=[np.number]).columns)}")
print(f"Categorical columns: {len(df.select_dtypes(include=['object']).columns)}")

print("\n✅ Initial data loading completed successfully!")
print("Next: We'll analyze our target variable (SalePrice) in detail")

### 📊 Data Loading and Initial Inspection

Loading the Ames Housing dataset and performing initial data quality checks. This step is crucial because it helps us understand:

- **Dataset size**: Number of rows and columns
- **Memory usage**: Important for large datasets
- **Data integrity**: Checking for duplicates
- **Basic structure**: Getting familiar with the data format

The `display()` function shows the first few rows in a formatted table, giving us a quick preview of our features and data types.

## 2. Initial Data Exploration

### 🎯 Target Variable Analysis

Understanding our target variable (SalePrice) is fundamental to building an effective regression model. This analysis helps us:

- **Identify the data distribution**: Is it normal, skewed, or has outliers?
- **Detect transformation needs**: Skewed data often benefits from log transformation
- **Understand the scale**: Price ranges help us set realistic expectations
- **Check for data quality issues**: Missing values, extreme outliers, etc.

The visualizations show:
1. **Histogram**: Overall distribution shape
2. **Box plot**: Outliers and quartiles
3. **Q-Q plot**: How closely the data follows a normal distribution
4. **Log-transformed histogram**: Shows if log transformation improves normality

In [ ]:
# Comprehensive analysis of our target variable: SalePrice
# This is the variable we want to predict, so understanding it deeply is crucial
target = 'SalePrice'

print("=== TARGET VARIABLE ANALYSIS: SalePrice ===")
print(f"Target variable: {target}")

# Calculate descriptive statistics to understand the distribution
print(f"\n📊 DESCRIPTIVE STATISTICS:")
target_stats = df[target].describe()
print(target_stats)

# Calculate distribution characteristics
# Skewness: measures asymmetry (0 = symmetric, >0 = right-skewed, <0 = left-skewed)
# Kurtosis: measures tail heaviness (3 = normal distribution)
skewness = df[target].skew()
kurtosis = df[target].kurtosis()
missing_count = df[target].isnull().sum()

print(f"\n📈 DISTRIBUTION CHARACTERISTICS:")
print(f"Skewness: {skewness:.4f} {'(right-skewed)' if skewness > 0.5 else '(approximately symmetric)' if abs(skewness) < 0.5 else '(left-skewed)'}")
print(f"Kurtosis: {kurtosis:.4f} {'(heavy tails)' if kurtosis > 3 else '(light tails)' if kurtosis < 3 else '(normal tails)'}")
print(f"Missing values: {missing_count} ({missing_count/len(df)*100:.1f}%)")

# Create comprehensive visualizations to understand the distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('SalePrice Distribution Analysis', fontsize=16, y=1.02)

# 1. Histogram - shows the overall shape of the distribution
axes[0,0].hist(df[target], bins=50, edgecolor='black', alpha=0.7, color='skyblue')
axes[0,0].set_title('SalePrice Distribution\n(Original Scale)')
axes[0,0].set_xlabel('Sale Price ($)')
axes[0,0].set_ylabel('Frequency')
axes[0,0].axvline(df[target].mean(), color='red', linestyle='--', label=f'Mean: ${df[target].mean():,.0f}')
axes[0,0].axvline(df[target].median(), color='orange', linestyle='--', label=f'Median: ${df[target].median():,.0f}')
axes[0,0].legend()

# 2. Box plot - excellent for identifying outliers and quartiles
box_data = axes[0,1].boxplot(df[target])
axes[0,1].set_title('SalePrice Box Plot\n(Outlier Detection)')
axes[0,1].set_ylabel('Sale Price ($)')
axes[0,1].grid(True, alpha=0.3)

# 3. Q-Q plot - compares our data to a theoretical normal distribution
stats.probplot(df[target], dist="norm", plot=axes[1,0])
axes[1,0].set_title('Q-Q Plot vs Normal Distribution\n(Normality Check)')
axes[1,0].grid(True, alpha=0.3)

# 4. Log-transformed distribution - often improves normality for price data
log_prices = np.log1p(df[target])  # log1p = log(1+x) to handle zeros safely
axes[1,1].hist(log_prices, bins=50, edgecolor='black', alpha=0.7, color='lightgreen')
axes[1,1].set_title('Log-Transformed SalePrice\n(Improved Normality)')
axes[1,1].set_xlabel('Log(Sale Price)')
axes[1,1].set_ylabel('Frequency')
axes[1,1].axvline(log_prices.mean(), color='red', linestyle='--', label=f'Mean: {log_prices.mean():.2f}')
axes[1,1].legend()

plt.tight_layout()
plt.show()

# Compare skewness before and after log transformation
log_skewness = log_prices.skew()
print(f"\n🔄 TRANSFORMATION IMPACT:")
print(f"Original skewness: {skewness:.4f}")
print(f"Log-transformed skewness: {log_skewness:.4f}")
print(f"Improvement: {'✅ Significant' if abs(log_skewness) < abs(skewness) * 0.5 else '⚠️ Moderate' if abs(log_skewness) < abs(skewness) else '❌ None'}")

print(f"\n💡 KEY INSIGHTS:")
print(f"   • Price range: ${df[target].min():,.0f} to ${df[target].max():,.0f}")
print(f"   • Mean price: ${df[target].mean():,.0f}")
print(f"   • Median price: ${df[target].median():,.0f}")
print(f"   • Standard deviation: ${df[target].std():,.0f}")
if skewness > 0.5:
    print(f"   • Distribution is right-skewed → Log transformation recommended")
else:
    print(f"   • Distribution is approximately symmetric → Transformation may not be needed")

In [ ]:
# Comprehensive analysis of data types and missing values
# Understanding our feature landscape is essential for preprocessing and model selection
print("=== DATA TYPES AND MISSING VALUES ANALYSIS ===")

# Separate features by data type - this determines how we'll preprocess them
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

# Remove target variable from numerical columns for feature analysis
if target in numerical_cols:
    numerical_cols.remove(target)

print(f"\n📊 FEATURE TYPE BREAKDOWN:")
print(f"Numerical features: {len(numerical_cols)} columns")
print(f"Categorical features: {len(categorical_cols)} columns")
print(f"Total features (excluding target): {len(numerical_cols) + len(categorical_cols)}")

# Analyze missing values across all columns
# Create a comprehensive missing values report
missing_data = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df)) * 100,
    'Data_Type': df.dtypes
})

# Focus on columns with missing values
missing_data = missing_data[missing_data['Missing_Count'] > 0]
missing_data = missing_data.sort_values('Missing_Percentage', ascending=False)

print(f"\n🔍 MISSING VALUES SUMMARY:")
print(f"Columns with missing values: {len(missing_data)}")
print(f"Total missing values: {missing_data['Missing_Count'].sum():,}")
print(f"Overall missing percentage: {(missing_data['Missing_Count'].sum() / (len(df) * len(df.columns))) * 100:.2f}%")

if len(missing_data) > 0:
    print(f"\n📋 TOP COLUMNS WITH MISSING VALUES:")
    print("=" * 80)
    for _, row in missing_data.head(10).iterrows():
        col_name = row['Column'][:25].ljust(25)  # Truncate long names
        missing_pct = row['Missing_Percentage']
        missing_count = int(row['Missing_Count'])
        data_type = str(row['Data_Type'])
        
        # Add interpretation
        severity = "🔴 High" if missing_pct > 20 else "🟡 Medium" if missing_pct > 5 else "🟢 Low"
        
        print(f"  {col_name} | {missing_count:>4} ({missing_pct:>5.1f}%) | {data_type:<8} | {severity}")
    
    # Create visualization of missing values
    if len(missing_data) > 0:
        plt.figure(figsize=(12, 6))
        top_missing = missing_data.head(15)
        plt.barh(range(len(top_missing)), top_missing['Missing_Percentage'])
        plt.yticks(range(len(top_missing)), top_missing['Column'])
        plt.xlabel('Missing Percentage (%)')
        plt.title('Top 15 Features with Missing Values')
        plt.gca().invert_yaxis()
        
        # Add percentage labels
        for i, v in enumerate(top_missing['Missing_Percentage']):
            plt.text(v + 0.5, i, f'{v:.1f}%', va='center')
        
        plt.tight_layout()
        plt.show()
else:
    print("✅ Excellent! No missing values found in the dataset!")

# Analyze the distribution of missing values by data type
print(f"\n📈 MISSING VALUES BY DATA TYPE:")
if len(missing_data) > 0:
    missing_by_type = missing_data.groupby('Data_Type').agg({
        'Missing_Count': ['count', 'sum', 'mean'],
        'Missing_Percentage': 'mean'
    }).round(2)
    print(missing_by_type)
else:
    print("No missing values to analyze by type.")

print(f"\n💡 PREPROCESSING IMPLICATIONS:")
print(f"   • Numerical features ({len(numerical_cols)}): Ready for scaling and transformation")
print(f"   • Categorical features ({len(categorical_cols)}): Need encoding (one-hot, target, etc.)")
if len(missing_data) > 0:
    high_missing = len(missing_data[missing_data['Missing_Percentage'] > 20])
    print(f"   • High missing features ({high_missing}): Consider dropping or advanced imputation")
    print(f"   • Strategy needed: Domain-specific imputation for each feature type")
else:
    print(f"   • No missing values: Can proceed directly to feature engineering")

### 🔍 Data Types and Missing Values Analysis

Before building models, we need to understand our feature landscape:

**Why this analysis matters:**
- **Numerical features**: Can be used directly in most ML algorithms
- **Categorical features**: Need encoding (one-hot, target encoding, etc.)
- **Missing values**: Must be handled appropriately for each feature type

**Missing values strategy:**
- **Numerical**: Often filled with median (robust to outliers) or mean
- **Categorical**: Filled with mode (most frequent) or 'Unknown'/'None'
- **Domain knowledge**: Some missing values are actually meaningful (e.g., no garage = missing garage features)

This analysis guides our preprocessing pipeline and feature engineering strategy.

## 3. Feature Engineering

Feature engineering is often the most impactful part of the machine learning pipeline. Good features can make a simple model outperform a complex one with poor features.

**Our comprehensive approach includes:**

1. **Strategic Missing Value Handling**: Different strategies for different feature types
2. **Domain-Specific Feature Creation**: Combining existing features to create meaningful new ones
3. **Data Transformations**: Normalizing skewed distributions for better model performance

**Why Feature Engineering Matters:**
- **Improves Model Performance**: Better features → better predictions
- **Captures Domain Knowledge**: Meaningful combinations often outperform raw features  
- **Handles Data Quality Issues**: Proper missing value treatment prevents errors
- **Enables Better Model Interpretation**: Clear, meaningful features are easier to explain

Feature engineering is both an art and a science - we use statistical techniques guided by domain knowledge about real estate.

In [ ]:
# Create a copy for feature engineering
df_fe = df.copy()

print("=== FEATURE ENGINEERING ===")

# 1. Handle missing values strategically
print("\n1. Handling Missing Values...")

# For numerical features, fill with median or 0 based on context
numerical_fill_median = [col for col in numerical_cols if col in df_fe.columns and df_fe[col].isnull().sum() > 0]
for col in numerical_fill_median:
    if 'Area' in col or 'SF' in col:  # Area features can be 0
        df_fe[col].fillna(0, inplace=True)
    else:
        df_fe[col].fillna(df_fe[col].median(), inplace=True)

# For categorical features, fill with 'None' or mode
categorical_fill = [col for col in categorical_cols if col in df_fe.columns and df_fe[col].isnull().sum() > 0]
for col in categorical_fill:
    # Features that logically can be 'None'
    if any(keyword in col for keyword in ['Garage', 'Bsmt', 'Fireplace', 'Pool', 'Fence', 'Alley']):
        df_fe[col].fillna('None', inplace=True)
    else:
        df_fe[col].fillna(df_fe[col].mode()[0] if len(df_fe[col].mode()) > 0 else 'Unknown', inplace=True)

print(f"   ✅ Missing values handled")

# 2. Create new features
print("\n2. Creating New Features...")

# Total square footage
area_cols = [col for col in df_fe.columns if 'SF' in col and col != target]
if len(area_cols) > 0:
    df_fe['TotalSF'] = df_fe[area_cols].sum(axis=1)
    print(f"   ✅ TotalSF created from {len(area_cols)} area columns")

# House age and remodel age
if 'Year Built' in df_fe.columns and 'Yr Sold' in df_fe.columns:
    df_fe['HouseAge'] = df_fe['Yr Sold'] - df_fe['Year Built']
    print(f"   ✅ HouseAge created")

if 'Year Remod/Add' in df_fe.columns and 'Yr Sold' in df_fe.columns:
    df_fe['RemodAge'] = df_fe['Yr Sold'] - df_fe['Year Remod/Add']
    df_fe['WasRemodeled'] = (df_fe['Year Remod/Add'] != df_fe['Year Built']).astype(int)
    print(f"   ✅ RemodAge and WasRemodeled created")

# Total bathrooms
bathroom_cols = [col for col in df_fe.columns if 'Bath' in col]
if len(bathroom_cols) > 0:
    # Give half baths 0.5 weight
    df_fe['TotalBaths'] = 0
    for col in bathroom_cols:
        if 'Half' in col:
            df_fe['TotalBaths'] += df_fe[col] * 0.5
        else:
            df_fe['TotalBaths'] += df_fe[col]
    print(f"   ✅ TotalBaths created from {len(bathroom_cols)} bathroom columns")

# Quality score (combine overall quality and condition)
if 'Overall Qual' in df_fe.columns and 'Overall Cond' in df_fe.columns:
    df_fe['QualityScore'] = df_fe['Overall Qual'] * df_fe['Overall Cond']
    print(f"   ✅ QualityScore created")

# Garage score
garage_cols = [col for col in df_fe.columns if 'Garage' in col and df_fe[col].dtype in ['int64', 'float64']]
if len(garage_cols) > 0:
    df_fe['GarageScore'] = df_fe[garage_cols].sum(axis=1)
    print(f"   ✅ GarageScore created")

print(f"\nOriginal features: {df.shape[1]}")
print(f"After feature engineering: {df_fe.shape[1]}")
print(f"New features created: {df_fe.shape[1] - df.shape[1]}")

In [ ]:
# 3. Transform skewed features
print("=== FEATURE TRANSFORMATION ===")

# Update numerical columns list
numerical_cols = df_fe.select_dtypes(include=[np.number]).columns.tolist()
if target in numerical_cols:
    numerical_cols.remove(target)

# Find skewed features
skewed_features = []
for col in numerical_cols:
    if df_fe[col].dtype in ['int64', 'float64']:
        skewness = df_fe[col].skew()
        if abs(skewness) > 0.75:  # Threshold for significant skewness
            skewed_features.append((col, skewness))

print(f"\nFound {len(skewed_features)} skewed features:")
skewed_df = pd.DataFrame(skewed_features, columns=['Feature', 'Skewness']).sort_values('Skewness', key=abs, ascending=False)
print(skewed_df.head(10).to_string(index=False))

# Apply Box-Cox transformation to positive skewed features
print("\nApplying transformations...")
transformed_features = []

for col, skewness in skewed_features:
    if df_fe[col].min() > 0:  # Box-Cox requires positive values
        # Use Box-Cox transformation
        lam = boxcox_normmax(df_fe[col] + 1)  # Add 1 to handle zeros
        df_fe[col] = boxcox1p(df_fe[col], lam)
        transformed_features.append(col)
    elif skewness > 0:  # Positive skew, use log transformation
        df_fe[col] = np.log1p(df_fe[col])
        transformed_features.append(col)

print(f"✅ Transformed {len(transformed_features)} features")

# Transform target variable
df_fe['SalePrice_log'] = np.log1p(df_fe[target])
print(f"✅ Target variable log-transformed")
print(f"   Original skewness: {df_fe[target].skew():.4f}")
print(f"   Transformed skewness: {df_fe['SalePrice_log'].skew():.4f}")

## 4. Feature Selection

Feature selection is crucial for model performance and interpretability. We'll use multiple methods to identify the most important features.

In [ ]:
# Prepare data for feature selection
print("=== FEATURE SELECTION ===")

# Update column lists
numerical_cols = df_fe.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df_fe.select_dtypes(include=['object']).columns.tolist()

# Remove target variables from features
exclude_cols = [target, 'SalePrice_log', 'Order', 'PID']  # Remove ID columns
numerical_cols = [col for col in numerical_cols if col not in exclude_cols]

print(f"Numerical features: {len(numerical_cols)}")
print(f"Categorical features: {len(categorical_cols)}")

# 1. Correlation-based feature selection for numerical features
print("\n1. Correlation Analysis...")

# Calculate correlation with target
correlations = df_fe[numerical_cols + [target]].corr()[target].abs().sort_values(ascending=False)
high_corr_features = correlations[correlations > 0.3].drop(target).head(20)

print(f"Top 15 features by correlation with {target}:")
for feature, corr in high_corr_features.head(15).items():
    print(f"  {feature:25}: {corr:.4f}")

# Visualize top correlations
plt.figure(figsize=(12, 8))
high_corr_features.head(15).plot(kind='barh')
plt.title('Top 15 Features by Correlation with SalePrice')
plt.xlabel('Absolute Correlation')
plt.tight_layout()
plt.show()

In [ ]:
# 2. Handle categorical variables with encoding
print("\n2. Categorical Variable Encoding...")

# For categorical features, we'll use target encoding for high-cardinality features
# and one-hot encoding for low-cardinality features

df_encoded = df_fe.copy()
encoded_features = []

for col in categorical_cols:
    unique_count = df_fe[col].nunique()
    
    if unique_count > 10:  # High cardinality - use target encoding
        # Target encoding (mean of target for each category)
        target_mean = df_fe.groupby(col)[target].mean()
        df_encoded[f"{col}_encoded"] = df_fe[col].map(target_mean)
        encoded_features.append(f"{col}_encoded")
        print(f"  Target encoded: {col} ({unique_count} categories)")
    
    elif unique_count > 2:  # Medium cardinality - use one-hot encoding
        # One-hot encoding
        dummies = pd.get_dummies(df_fe[col], prefix=col, drop_first=True)
        df_encoded = pd.concat([df_encoded, dummies], axis=1)
        encoded_features.extend(dummies.columns.tolist())
        print(f"  One-hot encoded: {col} ({unique_count} categories -> {len(dummies.columns)} features)")
    
    else:  # Binary - use label encoding
        le = LabelEncoder()
        df_encoded[f"{col}_encoded"] = le.fit_transform(df_fe[col])
        encoded_features.append(f"{col}_encoded")
        print(f"  Label encoded: {col} ({unique_count} categories)")

# Remove original categorical columns
df_encoded.drop(columns=categorical_cols, inplace=True)

print(f"\n✅ Categorical encoding completed")
print(f"  Original categorical features: {len(categorical_cols)}")
print(f"  Encoded features created: {len(encoded_features)}")
print(f"  Total features after encoding: {df_encoded.shape[1]}")

In [ ]:
# 3. Statistical feature selection
print("\n3. Statistical Feature Selection...")

# Prepare feature matrix
feature_cols = [col for col in df_encoded.columns if col not in [target, 'SalePrice_log']]
X = df_encoded[feature_cols]
y = df_encoded[target]

# Handle any remaining missing values
X = X.fillna(X.median())

print(f"Feature matrix shape: {X.shape}")

# SelectKBest with f_regression
selector_f = SelectKBest(score_func=f_regression, k=50)  # Select top 50 features
X_selected_f = selector_f.fit_transform(X, y)
selected_features_f = X.columns[selector_f.get_support()].tolist()

print(f"\nF-regression selected features: {len(selected_features_f)}")
print("Top 10 by F-score:")
feature_scores = list(zip(selected_features_f, selector_f.scores_[selector_f.get_support()]))
feature_scores.sort(key=lambda x: x[1], reverse=True)
for feature, score in feature_scores[:10]:
    print(f"  {feature:30}: {score:.2f}")

# Tree-based feature selection
print("\n4. Tree-based Feature Importance...")
rf_selector = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_selector.fit(X, y)

# Get feature importances
feature_importance = list(zip(X.columns, rf_selector.feature_importances_))
feature_importance.sort(key=lambda x: x[1], reverse=True)

print("Top 10 by Random Forest importance:")
for feature, importance in feature_importance[:10]:
    print(f"  {feature:30}: {importance:.4f}")

# Select top features from Random Forest
rf_top_features = [feature for feature, _ in feature_importance[:50]]

# Combine selections
combined_features = list(set(selected_features_f + rf_top_features))
print(f"\n✅ Combined feature selection: {len(combined_features)} features")

In [ ]:
# Visualize feature importance
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# Random Forest Feature Importance
top_rf_features = feature_importance[:15]
features_rf = [f[0] for f in top_rf_features]
importances_rf = [f[1] for f in top_rf_features]

axes[0].barh(range(len(features_rf)), importances_rf)
axes[0].set_yticks(range(len(features_rf)))
axes[0].set_yticklabels(features_rf)
axes[0].set_xlabel('Importance')
axes[0].set_title('Top 15 Features - Random Forest Importance')
axes[0].invert_yaxis()

# F-regression scores
top_f_scores = feature_scores[:15]
features_f = [f[0] for f in top_f_scores]
scores_f = [f[1] for f in top_f_scores]

axes[1].barh(range(len(features_f)), scores_f)
axes[1].set_yticks(range(len(features_f)))
axes[1].set_yticklabels(features_f)
axes[1].set_xlabel('F-Score')
axes[1].set_title('Top 15 Features - F-Regression Score')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

# Create final feature set
final_features = combined_features
print(f"\n🎯 Final feature set: {len(final_features)} features")
print(f"Selected features: {final_features[:10]}...")

## 5. Model Selection and Training

We'll train multiple regression models and compare their performance using cross-validation.

In [ ]:
# Prepare final dataset for modeling
print("=== MODEL PREPARATION ===")

# Create final feature matrix
X_final = df_encoded[final_features]
y_final = df_encoded['SalePrice_log']  # Use log-transformed target

# Handle any remaining missing values
X_final = X_final.fillna(X_final.median())

print(f"Final dataset shape: {X_final.shape}")
print(f"Target shape: {y_final.shape}")
print(f"Missing values in features: {X_final.isnull().sum().sum()}")
print(f"Missing values in target: {y_final.isnull().sum()}")

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X_final, y_final, test_size=0.2, random_state=42, stratify=pd.cut(y_final, bins=5)
)

print(f"\nTrain set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

# Scale features for linear models
scaler = RobustScaler()  # Less sensitive to outliers than StandardScaler
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"✅ Data prepared for modeling")

In [ ]:
# Define models to test
print("=== MODEL SELECTION ===")

# Define models
models = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(random_state=42),
    'Lasso': Lasso(random_state=42),
    'ElasticNet': ElasticNet(random_state=42),
    'Random Forest': RandomForestRegressor(random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
    'Extra Trees': ExtraTreesRegressor(random_state=42, n_jobs=-1),
    'KNN': KNeighborsRegressor(n_jobs=-1),
    'SVR': SVR()
}

# Add XGBoost and LightGBM if available
if xgb_available:
    models['XGBoost'] = xgb.XGBRegressor(random_state=42, n_jobs=-1)

if lgb_available:
    models['LightGBM'] = lgb.LGBMRegressor(random_state=42, n_jobs=-1, verbose=-1)

print(f"\nTesting {len(models)} models:")
for name in models.keys():
    print(f"  • {name}")

# Models that need scaled features
scaled_models = ['Linear Regression', 'Ridge', 'Lasso', 'ElasticNet', 'KNN', 'SVR']

# Cross-validation function
def evaluate_model(model, X_train, y_train, model_name, cv_folds=5):
    """Evaluate model using cross-validation"""
    scores = cross_val_score(model, X_train, y_train, cv=cv_folds, 
                            scoring='neg_mean_squared_error', n_jobs=-1)
    rmse_scores = np.sqrt(-scores)
    
    return {
        'model_name': model_name,
        'mean_rmse': rmse_scores.mean(),
        'std_rmse': rmse_scores.std(),
        'min_rmse': rmse_scores.min(),
        'max_rmse': rmse_scores.max()
    }

# Evaluate all models
print("\nEvaluating models (5-fold CV)...")
model_results = []

for name, model in models.items():
    print(f"\n  Training {name}...")
    
    # Use scaled features for linear models
    if name in scaled_models:
        X_train_input = X_train_scaled
    else:
        X_train_input = X_train
    
    try:
        result = evaluate_model(model, X_train_input, y_train, name)
        model_results.append(result)
        print(f"    RMSE: {result['mean_rmse']:.4f} (+/- {result['std_rmse']:.4f})")
    except Exception as e:
        print(f"    ❌ Error: {e}")

print("\n" + "="*60)
print("MODEL COMPARISON RESULTS")
print("="*60)

In [ ]:
# Display results
results_df = pd.DataFrame(model_results)
results_df = results_df.sort_values('mean_rmse')

print("\nCross-Validation Results (RMSE on log-transformed prices):")
print(results_df.to_string(index=False, float_format='%.4f'))

# Visualize results
plt.figure(figsize=(12, 8))
y_pos = range(len(results_df))
plt.barh(y_pos, results_df['mean_rmse'], xerr=results_df['std_rmse'], 
         alpha=0.7, capsize=5)
plt.yticks(y_pos, results_df['model_name'])
plt.xlabel('RMSE (Cross-Validation)')
plt.title('Model Comparison - Cross-Validation RMSE')
plt.gca().invert_yaxis()

# Add value labels
for i, v in enumerate(results_df['mean_rmse']):
    plt.text(v + 0.001, i, f'{v:.4f}', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

# Select top 3 models for hyperparameter tuning
top_models = results_df.head(3)['model_name'].tolist()
print(f"\n🎯 Top 3 models selected for hyperparameter tuning:")
for i, model in enumerate(top_models, 1):
    rmse = results_df[results_df['model_name'] == model]['mean_rmse'].iloc[0]
    print(f"  {i}. {model}: {rmse:.4f} RMSE")

## 6. Hyperparameter Tuning

We'll perform hyperparameter tuning on the top-performing models to optimize their performance.

In [ ]:
print("=== HYPERPARAMETER TUNING ===")

# Define parameter grids for top models
param_grids = {
    'Ridge': {
        'alpha': [0.1, 1.0, 10.0, 100.0, 1000.0]
    },
    'Lasso': {
        'alpha': [0.0001, 0.001, 0.01, 0.1, 1.0]
    },
    'ElasticNet': {
        'alpha': [0.001, 0.01, 0.1, 1.0],
        'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]
    },
    'Random Forest': {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'Gradient Boosting': {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.05, 0.1, 0.15],
        'max_depth': [3, 4, 5],
        'min_samples_split': [2, 5, 10]
    },
    'Extra Trees': {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
}

# Add XGBoost and LightGBM parameters if available
if xgb_available:
    param_grids['XGBoost'] = {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.05, 0.1, 0.15],
        'max_depth': [3, 4, 5],
        'subsample': [0.8, 0.9, 1.0]
    }

if lgb_available:
    param_grids['LightGBM'] = {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.05, 0.1, 0.15],
        'max_depth': [3, 4, 5],
        'num_leaves': [31, 50, 100]
    }

# Tune hyperparameters for top models
tuned_models = {}
tuning_results = []

for model_name in top_models:
    if model_name not in param_grids:
        print(f"\n⚠️ No parameter grid defined for {model_name}, skipping...")
        continue
    
    print(f"\n🔧 Tuning {model_name}...")
    
    # Get the base model
    base_model = models[model_name]
    
    # Choose input data (scaled or not)
    if model_name in scaled_models:
        X_train_input = X_train_scaled
        X_test_input = X_test_scaled
    else:
        X_train_input = X_train
        X_test_input = X_test
    
    # Use RandomizedSearchCV for faster tuning (or GridSearchCV for exhaustive search)
    search = RandomizedSearchCV(
        base_model,
        param_grids[model_name],
        n_iter=20,  # Number of parameter combinations to try
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1,
        random_state=42,
        verbose=0
    )
    
    try:
        search.fit(X_train_input, y_train)
        
        # Store the best model
        tuned_models[model_name] = search.best_estimator_
        
        # Make predictions
        y_pred = search.best_estimator_.predict(X_test_input)
        
        # Calculate metrics
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        tuning_results.append({
            'model': model_name,
            'best_params': search.best_params_,
            'cv_score': -search.best_score_,
            'test_rmse': rmse,
            'test_mae': mae,
            'test_r2': r2
        })
        
        print(f"  ✅ Best CV RMSE: {np.sqrt(-search.best_score_):.4f}")
        print(f"  ✅ Test RMSE: {rmse:.4f}")
        print(f"  ✅ Test R²: {r2:.4f}")
        print(f"  Best params: {search.best_params_}")
        
    except Exception as e:
        print(f"  ❌ Error tuning {model_name}: {e}")

print(f"\n✅ Hyperparameter tuning completed for {len(tuned_models)} models")

## 7. Model Evaluation and Validation

Let's evaluate our tuned models comprehensively and select the best performer.

In [ ]:
print("=== MODEL EVALUATION ===")

# Display tuning results
if tuning_results:
    tuning_df = pd.DataFrame(tuning_results)
    tuning_df = tuning_df.sort_values('test_rmse')
    
    print("\nTuned Model Performance:")
    for _, row in tuning_df.iterrows():
        print(f"\n{row['model']}:")
        print(f"  CV RMSE: {row['cv_score']:.4f}")
        print(f"  Test RMSE: {row['test_rmse']:.4f}")
        print(f"  Test MAE: {row['test_mae']:.4f}")
        print(f"  Test R²: {row['test_r2']:.4f}")
    
    # Select best model
    best_model_name = tuning_df.iloc[0]['model']
    best_model = tuned_models[best_model_name]
    best_rmse = tuning_df.iloc[0]['test_rmse']
    best_r2 = tuning_df.iloc[0]['test_r2']
    
    print(f"\n🏆 BEST MODEL: {best_model_name}")
    print(f"   Test RMSE: {best_rmse:.4f}")
    print(f"   Test R²: {best_r2:.4f}")
    
else:
    print("No tuning results available. Using best model from initial evaluation.")
    best_model_name = top_models[0]
    best_model = models[best_model_name]
    
    # Train the best model
    if best_model_name in scaled_models:
        best_model.fit(X_train_scaled, y_train)
        y_pred = best_model.predict(X_test_scaled)
    else:
        best_model.fit(X_train, y_train)
        y_pred = best_model.predict(X_test)
    
    best_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    best_r2 = r2_score(y_test, y_pred)

In [ ]:
# Detailed evaluation of the best model
print(f"=== DETAILED EVALUATION: {best_model_name} ===")

# Make predictions
if best_model_name in scaled_models:
    y_train_pred = best_model.predict(X_train_scaled)
    y_test_pred = best_model.predict(X_test_scaled)
else:
    y_train_pred = best_model.predict(X_train)
    y_test_pred = best_model.predict(X_test)

# Calculate comprehensive metrics
def calculate_metrics(y_true, y_pred, set_name):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    # Convert back from log space for interpretable metrics
    y_true_orig = np.expm1(y_true)
    y_pred_orig = np.expm1(y_pred)
    
    rmse_orig = np.sqrt(mean_squared_error(y_true_orig, y_pred_orig))
    mae_orig = mean_absolute_error(y_true_orig, y_pred_orig)
    mape = np.mean(np.abs((y_true_orig - y_pred_orig) / y_true_orig)) * 100
    
    print(f"\n{set_name} Set Metrics:")
    print(f"  Log-space RMSE: {rmse:.4f}")
    print(f"  Log-space MAE: {mae:.4f}")
    print(f"  R² Score: {r2:.4f}")
    print(f"  Original RMSE: ${rmse_orig:,.0f}")
    print(f"  Original MAE: ${mae_orig:,.0f}")
    print(f"  MAPE: {mape:.2f}%")
    
    return {
        'rmse_log': rmse, 'mae_log': mae, 'r2': r2,
        'rmse_orig': rmse_orig, 'mae_orig': mae_orig, 'mape': mape
    }

train_metrics = calculate_metrics(y_train, y_train_pred, "Training")
test_metrics = calculate_metrics(y_test, y_test_pred, "Test")

# Check for overfitting
rmse_diff = test_metrics['rmse_log'] - train_metrics['rmse_log']
r2_diff = train_metrics['r2'] - test_metrics['r2']

print(f"\nOverfitting Analysis:")
print(f"  RMSE difference (test - train): {rmse_diff:.4f}")
print(f"  R² difference (train - test): {r2_diff:.4f}")

if rmse_diff > 0.02 or r2_diff > 0.05:
    print(f"  ⚠️ Potential overfitting detected")
else:
    print(f"  ✅ No significant overfitting")

In [ ]:
# Visualize predictions
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Actual vs Predicted (log space)
axes[0,0].scatter(y_test, y_test_pred, alpha=0.6)
axes[0,0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0,0].set_xlabel('Actual (log)')
axes[0,0].set_ylabel('Predicted (log)')
axes[0,0].set_title('Actual vs Predicted (Log Space)')
axes[0,0].grid(True, alpha=0.3)

# 2. Actual vs Predicted (original space)
y_test_orig = np.expm1(y_test)
y_test_pred_orig = np.expm1(y_test_pred)
axes[0,1].scatter(y_test_orig, y_test_pred_orig, alpha=0.6)
axes[0,1].plot([y_test_orig.min(), y_test_orig.max()], [y_test_orig.min(), y_test_orig.max()], 'r--', lw=2)
axes[0,1].set_xlabel('Actual Price ($)')
axes[0,1].set_ylabel('Predicted Price ($)')
axes[0,1].set_title('Actual vs Predicted (Original Scale)')
axes[0,1].grid(True, alpha=0.3)

# 3. Residuals plot
residuals = y_test - y_test_pred
axes[1,0].scatter(y_test_pred, residuals, alpha=0.6)
axes[1,0].axhline(y=0, color='r', linestyle='--')
axes[1,0].set_xlabel('Predicted (log)')
axes[1,0].set_ylabel('Residuals')
axes[1,0].set_title('Residuals Plot')
axes[1,0].grid(True, alpha=0.3)

# 4. Residuals histogram
axes[1,1].hist(residuals, bins=30, edgecolor='black', alpha=0.7)
axes[1,1].set_xlabel('Residuals')
axes[1,1].set_ylabel('Frequency')
axes[1,1].set_title('Residuals Distribution')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Residual statistics
print(f"\nResidual Analysis:")
print(f"  Mean residual: {residuals.mean():.6f}")
print(f"  Std residual: {residuals.std():.4f}")
print(f"  Residual skewness: {residuals.skew():.4f}")
print(f"  Residual kurtosis: {residuals.kurtosis():.4f}")

## 8. Model Interpretation

Understanding what drives our model's predictions is crucial for business insights and model trust.

In [ ]:
print(f"=== MODEL INTERPRETATION: {best_model_name} ===")

# Feature importance analysis
if hasattr(best_model, 'feature_importances_'):
    # Tree-based models
    print("\n🌳 Tree-based Feature Importance:")
    
    feature_names = X_final.columns[X_final.columns.isin(final_features)]
    importances = best_model.feature_importances_
    
    # Create feature importance dataframe
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': importances
    }).sort_values('importance', ascending=False)
    
    print("\nTop 15 Most Important Features:")
    for _, row in importance_df.head(15).iterrows():
        print(f"  {row['feature']:30}: {row['importance']:.4f}")
    
    # Visualize feature importance
    plt.figure(figsize=(12, 8))
    top_features = importance_df.head(20)
    plt.barh(range(len(top_features)), top_features['importance'])
    plt.yticks(range(len(top_features)), top_features['feature'])
    plt.xlabel('Feature Importance')
    plt.title(f'Top 20 Feature Importance - {best_model_name}')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

elif hasattr(best_model, 'coef_'):
    # Linear models
    print("\n📈 Linear Model Coefficients:")
    
    feature_names = X_final.columns[X_final.columns.isin(final_features)]
    coefficients = best_model.coef_
    
    # Create coefficient dataframe
    coef_df = pd.DataFrame({
        'feature': feature_names,
        'coefficient': coefficients,
        'abs_coefficient': np.abs(coefficients)
    }).sort_values('abs_coefficient', ascending=False)
    
    print("\nTop 15 Features by Coefficient Magnitude:")
    for _, row in coef_df.head(15).iterrows():
        direction = "📈" if row['coefficient'] > 0 else "📉"
        print(f"  {direction} {row['feature']:28}: {row['coefficient']:8.4f}")
    
    # Visualize coefficients
    plt.figure(figsize=(12, 8))
    top_coefs = coef_df.head(20)
    colors = ['green' if x > 0 else 'red' for x in top_coefs['coefficient']]
    plt.barh(range(len(top_coefs)), top_coefs['coefficient'], color=colors, alpha=0.7)
    plt.yticks(range(len(top_coefs)), top_coefs['feature'])
    plt.xlabel('Coefficient Value')
    plt.title(f'Top 20 Feature Coefficients - {best_model_name}')
    plt.axvline(x=0, color='black', linestyle='-', alpha=0.5)
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

else:
    print("\n⚠️ Model doesn't support direct feature importance extraction.")
    print("Consider using permutation importance or SHAP values for interpretation.")

In [ ]:
# Business insights from top features
print("=== BUSINESS INSIGHTS ===")

if hasattr(best_model, 'feature_importances_') or hasattr(best_model, 'coef_'):
    if hasattr(best_model, 'feature_importances_'):
        top_features_list = importance_df.head(10)['feature'].tolist()
    else:
        top_features_list = coef_df.head(10)['feature'].tolist()
    
    print("\n🏠 Key Price Drivers Identified:")
    
    # Categorize features
    categories = {
        'Size/Area': [],
        'Quality/Condition': [],
        'Age/Time': [],
        'Location': [],
        'Amenities': [],
        'Other': []
    }
    
    for feature in top_features_list:
        feature_lower = feature.lower()
        if any(word in feature_lower for word in ['sf', 'area', 'size', 'total']):
            categories['Size/Area'].append(feature)
        elif any(word in feature_lower for word in ['qual', 'cond', 'quality', 'score']):
            categories['Quality/Condition'].append(feature)
        elif any(word in feature_lower for word in ['age', 'year', 'built', 'remod']):
            categories['Age/Time'].append(feature)
        elif any(word in feature_lower for word in ['neighborhood', 'location']):
            categories['Location'].append(feature)
        elif any(word in feature_lower for word in ['garage', 'bath', 'fireplace', 'pool']):
            categories['Amenities'].append(feature)
        else:
            categories['Other'].append(feature)
    
    for category, features in categories.items():
        if features:
            print(f"\n  {category}:")
            for feature in features:
                print(f"    • {feature}")
    
    print("\n💡 Actionable Insights:")
    print("  1. Focus on size/area features for maximum impact")
    print("  2. Quality improvements show strong returns")
    print("  3. Age/renovation timing affects value significantly")
    print("  4. Location remains a key factor (if applicable)")
    print("  5. Specific amenities drive premium pricing")

## 9. Model Persistence and Deployment Preparation

In [ ]:
print("=== MODEL PERSISTENCE ===")

import joblib
import json
from datetime import datetime

# Create model artifacts directory
import os
artifacts_dir = 'model_artifacts'
os.makedirs(artifacts_dir, exist_ok=True)

# Save the best model
model_filename = f'{artifacts_dir}/best_model_{best_model_name.lower().replace(" ", "_")}.joblib'
joblib.dump(best_model, model_filename)
print(f"✅ Model saved: {model_filename}")

# Save the scaler (if used)
if best_model_name in scaled_models:
    scaler_filename = f'{artifacts_dir}/scaler.joblib'
    joblib.dump(scaler, scaler_filename)
    print(f"✅ Scaler saved: {scaler_filename}")

# Save feature list
features_filename = f'{artifacts_dir}/selected_features.json'
with open(features_filename, 'w') as f:
    json.dump(final_features, f, indent=2)
print(f"✅ Features saved: {features_filename}")

# Save model metadata
metadata = {
    'model_name': best_model_name,
    'model_type': str(type(best_model).__name__),
    'training_date': datetime.now().isoformat(),
    'test_rmse_log': float(test_metrics['rmse_log']),
    'test_mae_log': float(test_metrics['mae_log']),
    'test_r2': float(test_metrics['r2']),
    'test_rmse_original': float(test_metrics['rmse_orig']),
    'test_mae_original': float(test_metrics['mae_orig']),
    'test_mape': float(test_metrics['mape']),
    'num_features': len(final_features),
    'training_samples': int(len(X_train)),
    'test_samples': int(len(X_test)),
    'uses_scaling': best_model_name in scaled_models,
    'target_transformation': 'log1p'
}

if hasattr(best_model, 'get_params'):
    try:
        metadata['model_parameters'] = best_model.get_params()
    except:
        metadata['model_parameters'] = 'Could not serialize parameters'

metadata_filename = f'{artifacts_dir}/model_metadata.json'
with open(metadata_filename, 'w') as f:
    json.dump(metadata, f, indent=2, default=str)
print(f"✅ Metadata saved: {metadata_filename}")

print(f"\n📦 Model artifacts saved in '{artifacts_dir}/' directory")
print(f"Files created:")
for file in os.listdir(artifacts_dir):
    print(f"  • {file}")

In [ ]:
# Create prediction function for deployment
print("=== DEPLOYMENT PREPARATION ===")

def predict_house_price(features_dict, model_path=None, scaler_path=None, features_path=None):
    """
    Predict house price using the trained model
    
    Parameters:
    features_dict: dict with feature names as keys and values
    model_path: path to saved model
    scaler_path: path to saved scaler (if needed)
    features_path: path to selected features list
    
    Returns:
    predicted price in original scale
    """
    
    # Load model and artifacts (in production, these would be loaded once at startup)
    if model_path:
        model = joblib.load(model_path)
    else:
        model = best_model
    
    if scaler_path and os.path.exists(scaler_path):
        scaler = joblib.load(scaler_path)
        use_scaling = True
    else:
        use_scaling = False
    
    if features_path:
        with open(features_path, 'r') as f:
            selected_features = json.load(f)
    else:
        selected_features = final_features
    
    # Create feature vector
    feature_vector = []
    for feature in selected_features:
        if feature in features_dict:
            feature_vector.append(features_dict[feature])
        else:
            # Handle missing features with median/mode
            feature_vector.append(0)  # Or use stored median values
    
    # Convert to numpy array and reshape
    X_input = np.array(feature_vector).reshape(1, -1)
    
    # Scale if needed
    if use_scaling:
        X_input = scaler.transform(X_input)
    
    # Make prediction
    y_pred_log = model.predict(X_input)[0]
    
    # Convert back to original scale
    y_pred_original = np.expm1(y_pred_log)
    
    return y_pred_original

# Example usage
print("\n🧪 Testing prediction function...")

# Get a sample from test set
sample_idx = 0
sample_features = dict(zip(final_features, X_test.iloc[sample_idx]))
actual_price = np.expm1(y_test.iloc[sample_idx])

# Make prediction
predicted_price = predict_house_price(sample_features)

print(f"\nSample Prediction:")
print(f"  Actual price: ${actual_price:,.0f}")
print(f"  Predicted price: ${predicted_price:,.0f}")
print(f"  Difference: ${abs(actual_price - predicted_price):,.0f}")
print(f"  Error rate: {abs(actual_price - predicted_price) / actual_price * 100:.2f}%")

print(f"\n✅ Prediction function ready for deployment")

## 10. Summary and Recommendations

In [ ]:
print("=== PROJECT SUMMARY ===")

print(f"\n🎯 FINAL MODEL PERFORMANCE:")
print(f"   Model: {best_model_name}")
print(f"   Test R²: {test_metrics['r2']:.4f} ({test_metrics['r2']*100:.1f}% variance explained)")
print(f"   Test RMSE: ${test_metrics['rmse_orig']:,.0f}")
print(f"   Test MAE: ${test_metrics['mae_orig']:,.0f}")
print(f"   Test MAPE: {test_metrics['mape']:.2f}%")

print(f"\n📊 DATASET INSIGHTS:")
print(f"   • Total samples: {df.shape[0]:,}")
print(f"   • Original features: {df.shape[1]}")
print(f"   • Engineered features: {df_fe.shape[1] - df.shape[1]}")
print(f"   • Selected features: {len(final_features)}")
print(f"   • Target transformation: Log(1+x) - reduced skewness from {df[target].skew():.2f} to {df_fe['SalePrice_log'].skew():.2f}")

print(f"\n🔧 FEATURE ENGINEERING HIGHLIGHTS:")
new_features = ['TotalSF', 'HouseAge', 'RemodAge', 'WasRemodeled', 'TotalBaths', 'QualityScore', 'GarageScore']
for feature in new_features:
    if feature in df_fe.columns:
        print(f"   ✅ {feature}: Created successfully")

print(f"\n🏆 MODEL SELECTION PROCESS:")
print(f"   • Models tested: {len(models)}")
print(f"   • Best performer: {best_model_name}")
print(f"   • Hyperparameter tuning: {'✅ Completed' if tuned_models else '⚠️ Skipped'}")
print(f"   • Cross-validation: 5-fold CV used")

print(f"\n💡 KEY RECOMMENDATIONS:")
print(f"   1. 📈 Model Performance: Achieving {test_metrics['r2']*100:.1f}% variance explanation is excellent")
print(f"   2. 🎯 Feature Focus: Size/area and quality features are primary value drivers")
print(f"   3. 🔄 Model Updates: Retrain quarterly with new market data")
print(f"   4. 📊 Monitoring: Track MAPE <{test_metrics['mape']:.0f}% for production performance")
print(f"   5. 🚀 Deployment: Model ready with artifacts in '{artifacts_dir}/' directory")

print(f"\n⚠️ LIMITATIONS & CONSIDERATIONS:")
print(f"   • Model assumes current market conditions")
print(f"   • Feature distributions may change over time")
print(f"   • Predictions are estimates with ±${test_metrics['mae_orig']:,.0f} typical error")
print(f"   • Regular retraining recommended for market changes")

print(f"\n🎉 REGRESSION MODELING PROJECT COMPLETED SUCCESSFULLY!")
print(f"\nNext steps:")
print(f"   • Deploy model to production environment")
print(f"   • Set up monitoring and alerting")
print(f"   • Establish retraining pipeline")
print(f"   • Create business dashboard for insights")